## RAG with langchain and Llama

---
1. Introduction

### Limitation of LLMs

- know nothing outside trainning data, e.g. up-to-date information, classified/private data
- not specialized in specific use cases
- tend to hallucinate confidently, possibly leading to missinformation
- produce black box output: do not clarify what has led to the generation of particular content

### Fine Tunning
- Enhances model Performance for specific use case through Transfer Learning with additional data.
- Changes model Parameters, enhancing speed and reducing Cost for specific Tasks
- Powerful tool for :
  - Incorporating non-dynamic or past data.
  - Specific Industries with nuances in writting style, and reducing costs for specific Tasks.
- Cut-off issue persists in absence of up-to-date Information

### Retrieval Augmented Generation
- Increases model capabilities through.
  - **Retrieving** external and up-to-date information.
  - **Augmenting** the original prompt given to the model.
  - **Generating** response using context plus Information.
- Ground LLM Model parameters remain unchanged (no Transfer Learning).
- Powerful Tool for making use of Dynamic up-to-date Information.
- White Box Output: Provides transparency behind the Model without Hallucination.